In [14]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pathlib
import os


In [15]:
img_dir = pathlib.Path("/Users/stefan_schmutz/Dropbox/stork-net_raw/images/2019/")

In [16]:
total_img = len(list(img_dir.glob("*.jpg")))

In [17]:
img_height = 480
img_width = 640
channels = 3

class_names = np.array(["3_stork", "1_stork", "0_stork", "2_stork"])

In [18]:
# The 1./255 is to convert from uint8 to float32 in range [0,1]
img_generator = ImageDataGenerator(rescale=1./255)

img_data_gen = img_generator.flow_from_directory(batch_size=1,
                                                 directory=img_dir,
                                                 shuffle=False,
                                                 target_size=(img_height, img_width),
                                                 class_mode="categorical",
                                                 classes = [""])

Found 69893 images belonging to 1 classes.


In [19]:
model = tf.keras.models.load_model("stork_net_v2.h5")

In [20]:
filenames = img_data_gen.filenames
nb_samples = len(filenames)

In [21]:
predictions = model.predict(img_data_gen)

# decode one-hot encoded labels
predictions_decoded = tf.argmax(predictions, axis=1)
predictions_label = [class_names[i] for i in predictions_decoded]

In [25]:
predictions_df = pd.DataFrame(list(zip(filenames, predictions, predictions_decoded.numpy(), predictions_label)),
                              columns =["filename", "prediction", "prediction_decoded", "prediction_label"]) 

In [26]:
predictions_df.head

<bound method NDFrame.head of                    filename  \
0      2019-01-01_00-05.jpg   
1      2019-01-01_00-10.jpg   
2      2019-01-01_00-15.jpg   
3      2019-01-01_00-20.jpg   
4      2019-01-01_00-25.jpg   
...                     ...   
69888  2019-08-31_23-35.jpg   
69889  2019-08-31_23-40.jpg   
69890  2019-08-31_23-45.jpg   
69891  2019-08-31_23-50.jpg   
69892  2019-08-31_23-55.jpg   

                                              prediction  prediction_decoded  \
0      [1.8295156e-14, 2.6989856e-06, 0.99999726, 2.2...                   2   
1      [4.573137e-14, 3.7987309e-06, 0.9999962, 7.274...                   2   
2      [6.2650656e-14, 1.813997e-05, 0.9999819, 6.346...                   2   
3      [9.6246484e-14, 8.814876e-06, 0.9999912, 2.258...                   2   
4      [9.2637025e-14, 9.0843905e-06, 0.99999094, 1.8...                   2   
...                                                  ...                 ...   
69888  [3.7124048e-10, 1.3933508e-05,

In [27]:
predictions_df.to_csv("predictions_2019.csv", index=False)